In [26]:
from dotenv import load_dotenv

_ = load_dotenv()

In [24]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [25]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [29]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [38]:
PLAN_PROMPT = """You are an expert disaster response planner tasked with creating a comprehensive \
    response plan for a given disaster scenario. Write such an outline for the user provided \
    disaster scenario. Give an outline of the plan along with any relevant notes or instructions \
    for the disaster response plan.
"""

In [37]:
RESPONDER_PROMPT = """You are a disaster response assistant tasked with executing a comprehensive disaster response plan. \
Utilize the detailed plan provided by the planning agent to implement the response effectively. Your responsibilities include \
coordinating resources, managing communication, and adapting to evolving situations. If there are updates or critiques from \
the planning agent or response teams, adjust your actions accordingly. Utilize all the information below as needed:

------

{content}"""

In [32]:
REFLECTION_PROMPT = """You are a disaster response expert reviewing a disaster response plan. 
Generate critique and recommendations for the provided plan. 
Provide detailed recommendations, including requests for additional details, improvements in strategy, and suggestions for better resource management and communication.
Ensure your feedback is constructive and actionable.

------

{content}"""

In [33]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can 
be used to enhance the following disaster response plan. Generate a list of search queries that will gather 
any relevant information. Only generate 3 queries max. 

------

{content}"""

In [34]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can 
be used to make any requested revisions to the disaster response plan (as outlined below). 
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max. 

------

{content}"""